# data cleaning

This notebook processes 550 project data

## Setup

Run the cell below to import necesary modules 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

import google drive

In [ ]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)

Mounted at /content/drive


opening correct paths in G Drive. `dla-1033-transfers.csv` is data post Feruguson after Trump rescinds Obama order to limit miliary police. The file `leso_raw.csv` is pre Obama's order.  This decribes the orders: https://www.reuters.com/article/us-usa-police-equipment-idUSKCN1B81TI


In [ ]:
pre_ferg_path = '/content/drive/MyDrive/Classes/550 Databases/Project/datasets/leso_raw.csv' # Path to cars dataset in your drive here
pre_ferg = pd.read_csv(pre_ferg_path)
pre_ferg.head()

,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date,PSC NAME
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1.0,Each,499.0,D,1.0,2012-08-30,"GUNS, THROUGH 30MM"
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1.0,Each,499.0,D,1.0,2012-08-30,"GUNS, THROUGH 30MM"
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1.0,Each,499.0,D,1.0,2012-08-30,"GUNS, THROUGH 30MM"
3,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1.0,Each,499.0,D,1.0,2012-08-30,"GUNS, THROUGH 30MM"
4,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",1.0,Each,499.0,D,1.0,2012-08-30,"GUNS, THROUGH 30MM"


In [ ]:
post_ferg_path = '/content/drive/MyDrive/Classes/550 Databases/Project/datasets/dla-1033-transfers.csv' # Path to cars dataset in your drive here
post_ferg = pd.read_csv(post_ferg_path)
post_ferg.head()

,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date,Station Type
0,AL,ABBEVILLE POLICE DEPT,2320-01-371-9584,"TRUCK,UTILITY",1,Each,62627.0,C,1.0,2016-09-29 00:00:00.000,State
1,AL,ABBEVILLE POLICE DEPT,1005-01-587-7175,"MOUNT,RIFLE",10,Each,1626.0,D,1.0,2016-09-19 00:00:00.000,State
2,AL,ABBEVILLE POLICE DEPT,2320-01-371-9584,"TRUCK,UTILITY",1,Each,62627.0,C,1.0,2016-09-29 00:00:00.000,State
3,AL,ABBEVILLE POLICE DEPT,2355-01-553-4634,MINE RESISTANT VEHICLE,1,Each,658000.0,C,1.0,2016-11-09 00:00:00.000,State
4,AL,ABBEVILLE POLICE DEPT,1240-01-411-1265,"SIGHT,REFLEX",9,Each,321.0,D,1.0,2016-09-14 00:00:00.000,State


Finally, run the cell below to initialize several functions that will spot-check the correctness of your solutions as you complete the exercises.

Store the number of rows and columns in the dataset in variables named `row_count` and `col_count` respectively.

In [ ]:
pf_row_count = pre_ferg.shape[0]
pf_col_count = pre_ferg.shape[1]

print('pre-ferg data')
print('PreF Number of rows: {}'.format(pf_row_count))
print('PreF Number of columns: {}'.format(pf_col_count))

print(30*'-')
print('post-ferg data')
pof_row_count = post_ferg.shape[0]
pof_col_count = post_ferg.shape[1]

print('POF Number of rows: {}'.format(pof_row_count))
print('POF Number of columns: {}'.format(pof_col_count))

pre-ferg data
PreF Number of rows: 194792
PreF Number of columns: 11
------------------------------
post-ferg data
POF Number of rows: 141068
POF Number of columns: 11


Output the datatype of each column. Which of these surprises you, based on your beliefs about what each column represents? 

In [ ]:

print(pre_ferg.dtypes)
print('-'*30)
print(post_ferg.dtypes)

# Suprising: Qauntity is float vs. int. Change to int

State                  object
Station Name (LEA)     object
NSN                    object
Item Name              object
Quantity              float64
UI                     object
Acquisition Value     float64
DEMIL Code             object
DEMIL IC              float64
Ship Date              object
PSC NAME               object
dtype: object
------------------------------
State                  object
Station Name (LEA)     object
NSN                    object
Item Name              object
Quantity                int64
UI                     object
Acquisition Value     float64
DEMIL Code             object
DEMIL IC              float64
Ship Date              object
Station Type           object
dtype: object


In [ ]:
#  change datatype of Qauntity column to int for consistency
pre_ferg.Quantity = pre_ferg.Quantity.astype(int)
pre_ferg.dtypes

State                  object
Station Name (LEA)     object
NSN                    object
Item Name              object
Quantity                int64
UI                     object
Acquisition Value     float64
DEMIL Code             object
DEMIL IC              float64
Ship Date              object
PSC NAME               object
dtype: object

Store the size of each column in KB in a series 

In [ ]:
pre_ferg_size = pre_ferg.memory_usage(deep=True).sort_values(ascending=False) / 1e3
post_ferg_size = post_ferg.memory_usage(deep=True).sort_values(ascending=False) / 1e3
print("pre_ferg: ", pre_ferg_size)
print('-'*30)
print("post_ferg: ", post_ferg_size)

pre_ferg:  PSC NAME              16578.840
Station Name (LEA)    15795.334
Item Name             15159.594
NSN                   14219.352
Ship Date             13051.064
UI                    11883.139
State                 11492.728
DEMIL Code            11297.936
DEMIL IC               1558.336
Acquisition Value      1558.336
Quantity               1558.336
Index                     0.128
dtype: float64
------------------------------
post_ferg:  Station Name (LEA)    11497.899
Ship Date             11285.440
Item Name             10982.068
NSN                   10297.956
Station Type           8746.216
UI                     8606.049
State                  8323.012
DEMIL Code             8181.944
DEMIL IC               1128.544
Acquisition Value      1128.544
Quantity               1128.544
Index                     0.128
dtype: float64


Store the total size of the dataset in MB in a variable named `cars_mb`. Again, include the sizes of the objects that the columns contain. 



In [ ]:
pre_mb = pre_ferg.memory_usage(deep=True).sum() / 1e6
post_mb = post_ferg.memory_usage(deep=True).sum() / 1e6
print("pre_ferg: ", pre_mb)
print('-'*30)
print("post_ferg: ", post_mb)

pre_ferg:  114.153123
------------------------------
post_ferg:  81.306344


## Missing Values

Which columns have missing values? *Consider all possible representations of missing values*

*Hint: You should find 10 columns with missing values*

In [ ]:
# create a list of numeric columns and a list of text columns

def misVals(dataset):
    numeric_cols, categorical_cols = list(), list()
    for col in dataset.columns:
      if dataset[col].dtype in [np.float64, np.int64]:
        numeric_cols.append(col)
      else:
        categorical_cols.append(col)
    # create dataframes containing only numeric and categorical cols respectively
    data_num = dataset.loc[:, numeric_cols]
    data_cat = dataset.loc[:, categorical_cols]

    # find all numeric columns with missing values
    num_missing = list(data_num.columns[data_num.isna().any()])
    print("what columns have numbers missing", num_missing)

    # find all categorical columns with missing values
    # (Assuming missing placeholder contains a '?'. Not always a good assumption,
    # it works for UCI Machine Learning Repo Datasets usually.)
    cat_missing = list()
    for col in data_cat.columns:
      curr_vals = data_cat[col].unique()
      for val in curr_vals:
        if val == np.nan in val:
          cat_missing.append(col)
    print("what columns have strings missing", cat_missing)
    return(numeric_cols, categorical_cols,data_num, data_cat, num_missing, cat_missing)

In [ ]:


numeric_cols, categorical_cols, data_num, data_cat, num_missing, cat_missing = misVals(pre_ferg)

po_numeric_cols, po_categorical_cols, po_data_num, po_data_cat, po_num_missing, po_cat_missing = misVals(post_ferg)


what columns have numbers missing ['DEMIL IC']
what columns have strings missing []
what columns have numbers missing ['DEMIL IC']
what columns have strings missing []


Store the number of missing values in each column in descending order in a `Series` named `missing_count`.


In [ ]:
# compile list of placeholder values
def misCount(data):
    placeholders = [np.NaN]
    # for col in cat_missing:
    #   for val in cars_cat[col].unique():
    #     if '?' in val:
    #       placeholders.append(val)
    # remove duplicates
    placeholders = list(set(placeholders))
    missing = data.isin(placeholders)
    missing_count = missing.sum().sort_values(ascending=False)

    return missing_count, missing

In [ ]:
missing_count, missing = misCount(pre_ferg)
print(missing_count)
print(30*'_')
po_missing_count, po_missing = misCount(post_ferg)
print(po_missing_count)

DEMIL IC              18224
Item Name                 1
PSC NAME                  0
Ship Date                 0
DEMIL Code                0
Acquisition Value         0
UI                        0
Quantity                  0
NSN                       0
Station Name (LEA)        0
State                     0
dtype: int64
______________________________
DEMIL IC              8905
Station Type             0
Ship Date                0
DEMIL Code               0
Acquisition Value        0
UI                       0
Quantity                 0
Item Name                0
NSN                      0
Station Name (LEA)       0
State                    0
dtype: int64


Store the number of rows with at least one missing value one missing value in a variable named `row_missing_count`. And store the portion of rows with at least one missing value in a variable named `row_missing_frac`

In [ ]:
row_missing_count = missing.any(axis=1).sum()
row_missing_frac = missing.any(axis=1).sum() / po_missing.shape[0]
po_row_missing_count = po_missing.any(axis=1).sum()
po_row_missing_frac = po_missing.any(axis=1).sum() / po_missing.shape[0]

print('Number of rows with missing values: {}'.format(row_missing_count))
print('Fraction of rows with missing values: {}'.format(row_missing_frac))

print('-'*30)
print('Number of rows with missing values: {}'.format(po_row_missing_count))
print('Fraction of rows with missing values: {}'.format(po_row_missing_frac))

Number of rows with missing values: 18224
Fraction of rows with missing values: 0.12918592451867184
------------------------------
Number of rows with missing values: 8905
Fraction of rows with missing values: 0.06312558482434004


## remove null rows and columns


In [ ]:
pre_ferg_clean = pre_ferg.drop("DEMIL IC", axis=1)
post_ferg_clean = post_ferg.drop("DEMIL IC", axis=1)
pre_ferg_clean = pre_ferg_clean.dropna()
post_ferg_clean = post_ferg_clean.dropna()

In [ ]:
# check if cleaned
numeric_cols, categorical_cols, data_num, data_cat, num_missing, cat_missing = misVals(pre_ferg_clean)
po_numeric_cols, po_categorical_cols, po_data_num, po_data_cat, po_num_missing, po_cat_missing = misVals(post_ferg_clean)
missing_count, missing = misCount(pre_ferg_clean)
print(missing_count)
print(30*'_')
po_missing_count, po_missing = misCount(post_ferg_clean)
print(po_missing_count)

what columns have numbers missing []
what columns have strings missing []
what columns have numbers missing []
what columns have strings missing []
PSC NAME              0
Ship Date             0
DEMIL Code            0
Acquisition Value     0
UI                    0
Quantity              0
Item Name             0
NSN                   0
Station Name (LEA)    0
State                 0
dtype: int64
______________________________
Station Type          0
Ship Date             0
DEMIL Code            0
Acquisition Value     0
UI                    0
Quantity              0
Item Name             0
NSN                   0
Station Name (LEA)    0
State                 0
dtype: int64


# remove and combine duplicates 

In [ ]:
pre_ferg_clean.dtypes

State                  object
Station Name (LEA)     object
NSN                    object
Item Name              object
Quantity                int64
UI                     object
Acquisition Value     float64
DEMIL Code             object
Ship Date              object
PSC NAME               object
dtype: object

In [ ]:
# 

In [ ]:
# combine all duplicate rows and add there Quantity from PRE-FERG

rem_dup_pre = pre_ferg_clean.groupby(pre_ferg_clean.columns.tolist(),as_index=False).size()



rem_dup_pre['Quantity'] = rem_dup_pre['size'] - 1 + rem_dup_pre['Quantity'] 
rem_dup_pre = rem_dup_pre.drop(['size'], axis=1)
rem_dup_pre

,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,Ship Date,PSC NAME
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",7,Each,499.00,D,2006-05-19,"GUNS, THROUGH 30MM"
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",123,Each,499.00,D,2012-08-30,"GUNS, THROUGH 30MM"
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1385-01-574-4707,"ROBOT,EXPLOSIVE ORDNANCE DISPOSAL",1,Each,10000.00,D,2012-05-24,SURFACE USE EXPLOSIVE ORDNANCE DISPOSAL TOOLS ...
3,AK,ALASKA DEPT OF PUBLIC SAFETY,5820-01-564-4845,"PROCESSOR GROUP,SIGNAL DATA",2,Each,5000.00,D,2012-07-19,"RADIO AND TELEVISION COMMUNICATION EQUIPMENT, ..."
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,NIGHT VISION GOGGLE,8,Each,4300.00,F,1998-05-19,"NIGHT VISION EQUIPMENT, EMITTED AND REFLECTED ..."
...,...,...,...,...,...,...,...,...,...,...
83497,WY,WY STATE PARKS & HISTORICAL SITES,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",22,Each,138.00,D,2007-09-10,"GUNS, THROUGH 30MM"
83498,WY,WY STATE PARKS & HISTORICAL SITES,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",4,Each,138.00,D,2012-07-10,"GUNS, THROUGH 30MM"
83499,WY,WYOMING LIVE STOCK BOARD,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",7,Each,58.71,D,2010-06-11,"GUNS, THROUGH 30MM"
83500,WY,WYOMING WOMEN'S CENTER LUSK PRISON,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",5,Each,499.00,D,2012-06-21,"GUNS, THROUGH 30MM"


In [ ]:
# combine all duplicate rows and add there Quantity from POST FERG

rem_dup_po = post_ferg_clean.groupby(post_ferg_clean.columns.tolist(),as_index=False).size()


rem_dup_po['Quantity'] = rem_dup_po['size'] - 1 + rem_dup_po['Quantity'] 
rem_dup_po = rem_dup_po.drop(['size'], axis=1)
rem_dup_po

,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,Ship Date,Station Type
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",56,Each,499.00,D,1998-01-22 00:00:00.000,State
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",4,Each,499.00,D,2002-08-26 00:00:00.000,State
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",4,Each,138.00,D,2000-04-04 00:00:00.000,State
3,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-01-630-9508,"MAGAZINE,CARTRIDGE",8,Each,13.56,D,2017-04-04 00:00:00.000,State
4,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-01-630-9508,"MAGAZINE,CARTRIDGE",50,Each,13.56,D,2017-04-04 00:00:00.000,State
...,...,...,...,...,...,...,...,...,...,...
54173,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,D,2001-11-27 00:00:00.000,State
54174,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,D,2002-01-29 00:00:00.000,State
54175,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2310-01-111-2274,"TRUCK,AMBULANCE",1,Each,96466.00,C,2019-09-25 00:00:00.000,State
54176,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,"TRUCK,UTILITY",1,Each,192513.00,C,2018-03-12 00:00:00.000,State


In [ ]:
# # Create the groupby_dict  
# groupby_dict = {"State":"state", 
#            "Station Name (LEA)":"station_name", 
#            "NSN":"NSN", 
#            "Item Name":"item_name",
#            "Ship Date": "ship_date"} 
  
# # df = df.set_index('ID') 
# new_group = rem_dup_pre.groupby(groupby_dict, axis = 1).sum() 
# print(new_group) 
# new_group.to_csv("new_group.csv", index=True)
# # rem_dup_po.to_csv("rempo.csv", index=True)
# # rem_dup_po.to_csv("postFerg.csv", index=False)
# # concat_mil.index.name = "id"
# # census.to_csv("census.csv", index=True)

# from google.colab import files
# files.download('new_group.csv')

In [ ]:
# df = rem_dup_pre.groupby(['State','Station Name (LEA)','NSN']).size()

# # new_group = rem_dup_pre.groupby(groupby_dict, axis = 1).sum() 

# df.to_csv("new_group.csv", index=True)
# # rem_dup_po.to_csv("rempo.csv", index=True)
# # rem_dup_po.to_csv("postFerg.csv", index=False)
# # concat_mil.index.name = "id"
# # census.to_csv("census.csv", index=True)

# from google.colab import files
# files.download('new_group.csv')

In [ ]:
rem_dup_pre

,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,Ship Date,PSC NAME
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",7,Each,499.00,D,2006-05-19,"GUNS, THROUGH 30MM"
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",123,Each,499.00,D,2012-08-30,"GUNS, THROUGH 30MM"
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1385-01-574-4707,"ROBOT,EXPLOSIVE ORDNANCE DISPOSAL",1,Each,10000.00,D,2012-05-24,SURFACE USE EXPLOSIVE ORDNANCE DISPOSAL TOOLS ...
3,AK,ALASKA DEPT OF PUBLIC SAFETY,5820-01-564-4845,"PROCESSOR GROUP,SIGNAL DATA",2,Each,5000.00,D,2012-07-19,"RADIO AND TELEVISION COMMUNICATION EQUIPMENT, ..."
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,NIGHT VISION GOGGLE,8,Each,4300.00,F,1998-05-19,"NIGHT VISION EQUIPMENT, EMITTED AND REFLECTED ..."
...,...,...,...,...,...,...,...,...,...,...
83497,WY,WY STATE PARKS & HISTORICAL SITES,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",22,Each,138.00,D,2007-09-10,"GUNS, THROUGH 30MM"
83498,WY,WY STATE PARKS & HISTORICAL SITES,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",4,Each,138.00,D,2012-07-10,"GUNS, THROUGH 30MM"
83499,WY,WYOMING LIVE STOCK BOARD,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",7,Each,58.71,D,2010-06-11,"GUNS, THROUGH 30MM"
83500,WY,WYOMING WOMEN'S CENTER LUSK PRISON,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",5,Each,499.00,D,2012-06-21,"GUNS, THROUGH 30MM"


In [ ]:
# add a total cost of aquisition and an item cost
def addTot(dataset):
  dataset['Total Purchase Cost'] = 0
  for i in range(len(dataset)):
    if dataset.iloc[i,5] == 'Each':
      dataset.iloc[i,10] = dataset.iloc[i, 4]*dataset.iloc[i,6]
    else:
      dataset.iloc[i,10] = dataset.iloc[i,6]

In [ ]:
addTot(rem_dup_pre)
addTot(rem_dup_po)

In [ ]:
print(rem_dup_pre)
print('-'*30)
print(rem_dup_po)

      State  ... Total Purchase Cost
0        AK  ...             3493.00
1        AK  ...            61377.00
2        AK  ...            10000.00
3        AK  ...            10000.00
4        AK  ...            34400.00
...     ...  ...                 ...
83497    WY  ...             3036.00
83498    WY  ...              552.00
83499    WY  ...              410.97
83500    WY  ...             2495.00
83501    WY  ...             3600.00

[83502 rows x 11 columns]
------------------------------
      State                Station Name (LEA)  ... Station Type Total Purchase Cost
0        AK      ALASKA DEPT OF PUBLIC SAFETY  ...        State            27944.00
1        AK      ALASKA DEPT OF PUBLIC SAFETY  ...        State             1996.00
2        AK      ALASKA DEPT OF PUBLIC SAFETY  ...        State              552.00
3        AK      ALASKA DEPT OF PUBLIC SAFETY  ...        State              108.48
4        AK      ALASKA DEPT OF PUBLIC SAFETY  ...        State              6

In [ ]:
rem_dup_po

,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,Ship Date,Station Type,Total Purchase Cost
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",56,Each,499.00,D,1998-01-22 00:00:00.000,State,27944.00
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",4,Each,499.00,D,2002-08-26 00:00:00.000,State,1996.00
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",4,Each,138.00,D,2000-04-04 00:00:00.000,State,552.00
3,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-01-630-9508,"MAGAZINE,CARTRIDGE",8,Each,13.56,D,2017-04-04 00:00:00.000,State,108.48
4,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-01-630-9508,"MAGAZINE,CARTRIDGE",50,Each,13.56,D,2017-04-04 00:00:00.000,State,678.00
...,...,...,...,...,...,...,...,...,...,...,...
54173,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,D,2001-11-27 00:00:00.000,State,234.84
54174,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,D,2002-01-29 00:00:00.000,State,234.84
54175,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2310-01-111-2274,"TRUCK,AMBULANCE",1,Each,96466.00,C,2019-09-25 00:00:00.000,State,96466.00
54176,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,"TRUCK,UTILITY",1,Each,192513.00,C,2018-03-12 00:00:00.000,State,192513.00


In [ ]:
rem_dup_pre

,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,Ship Date,PSC NAME,Total Purchase Cost
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",7,Each,499.00,D,2006-05-19,"GUNS, THROUGH 30MM",3493.00
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",123,Each,499.00,D,2012-08-30,"GUNS, THROUGH 30MM",61377.00
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1385-01-574-4707,"ROBOT,EXPLOSIVE ORDNANCE DISPOSAL",1,Each,10000.00,D,2012-05-24,SURFACE USE EXPLOSIVE ORDNANCE DISPOSAL TOOLS ...,10000.00
3,AK,ALASKA DEPT OF PUBLIC SAFETY,5820-01-564-4845,"PROCESSOR GROUP,SIGNAL DATA",2,Each,5000.00,D,2012-07-19,"RADIO AND TELEVISION COMMUNICATION EQUIPMENT, ...",10000.00
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,NIGHT VISION GOGGLE,8,Each,4300.00,F,1998-05-19,"NIGHT VISION EQUIPMENT, EMITTED AND REFLECTED ...",34400.00
...,...,...,...,...,...,...,...,...,...,...,...
83497,WY,WY STATE PARKS & HISTORICAL SITES,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",22,Each,138.00,D,2007-09-10,"GUNS, THROUGH 30MM",3036.00
83498,WY,WY STATE PARKS & HISTORICAL SITES,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",4,Each,138.00,D,2012-07-10,"GUNS, THROUGH 30MM",552.00
83499,WY,WYOMING LIVE STOCK BOARD,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",7,Each,58.71,D,2010-06-11,"GUNS, THROUGH 30MM",410.97
83500,WY,WYOMING WOMEN'S CENTER LUSK PRISON,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",5,Each,499.00,D,2012-06-21,"GUNS, THROUGH 30MM",2495.00


# merge the two datasets into one

# download as CSV 

In [ ]:

# rem_dup_pre.to_csv("preFerg.csv", index=False)
# rem_dup_po.to_csv("postFerg.csv", index=False)
# # census.index.name = "id"
# # census.to_csv("census.csv", index=True)

# from google.colab import files
# files.download('preFerg.csv') 
# files.download("postFerg.csv")

In [ ]:
# from google.colab import files
# files.download('preFerg.csv') 
# files.download("postFerg.csv")

In [ ]:
# rem_dup_po.dtypes

In [ ]:
# Before merge check if any NULL values
print(rem_dup_po.shape, rem_dup_pre.shape)
print(rem_dup_pre.isnull().values.any())
print(rem_dup_po.isnull().values.any())

(54178, 11) (83502, 11)
False
False


In [ ]:

# for i in range(len(rem_dup_po)):
#   # rem_dup_pre = rem_dup_pre.append(rem_dup_pre.iloc[i,:])
#   rem_dup_pre = pd.concat([rem_dup_pre,rem_dup_pre.iloc[i,:]]).reset_index(drop=True)

In [ ]:
rem_dup_po.dtypes

State                   object
Station Name (LEA)      object
NSN                     object
Item Name               object
Quantity                 int64
UI                      object
Acquisition Value      float64
DEMIL Code              object
Ship Date               object
Station Type            object
Total Purchase Cost    float64
dtype: object

In [ ]:
rem_dup_pre.dtypes

State                   object
Station Name (LEA)      object
NSN                     object
Item Name               object
Quantity                 int64
UI                      object
Acquisition Value      float64
DEMIL Code              object
Ship Date               object
PSC NAME                object
Total Purchase Cost    float64
dtype: object

In [ ]:
# merge the dataset

concat_mil = pd.concat([rem_dup_pre, rem_dup_po]).reset_index(drop=True)

In [ ]:
concat_mil= concat_mil.drop(['Station Type'], axis=1)
concat_mil = concat_mil.drop(['PSC NAME'], axis=1)

In [ ]:
concat_mil

,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,Ship Date,Total Purchase Cost
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",7,Each,499.00,D,2006-05-19,3493.00
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",123,Each,499.00,D,2012-08-30,61377.00
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1385-01-574-4707,"ROBOT,EXPLOSIVE ORDNANCE DISPOSAL",1,Each,10000.00,D,2012-05-24,10000.00
3,AK,ALASKA DEPT OF PUBLIC SAFETY,5820-01-564-4845,"PROCESSOR GROUP,SIGNAL DATA",2,Each,5000.00,D,2012-07-19,10000.00
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,NIGHT VISION GOGGLE,8,Each,4300.00,F,1998-05-19,34400.00
...,...,...,...,...,...,...,...,...,...,...
137675,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,D,2001-11-27 00:00:00.000,234.84
137676,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,D,2002-01-29 00:00:00.000,234.84
137677,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2310-01-111-2274,"TRUCK,AMBULANCE",1,Each,96466.00,C,2019-09-25 00:00:00.000,96466.00
137678,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,"TRUCK,UTILITY",1,Each,192513.00,C,2018-03-12 00:00:00.000,192513.00


In [ ]:
concat_mil.isna().any()

State                  False
Station Name (LEA)     False
NSN                    False
Item Name              False
Quantity               False
UI                     False
Acquisition Value      False
DEMIL Code             False
Ship Date              False
Total Purchase Cost    False
dtype: bool

Which columns have misentered values? 

*Hint: There are 2 columns with misentered values. One has an impossible min, and the other has an implausible max*

In [ ]:
concat_mil.describe()

# ANS : highway_mpg has impossible min (0),
# and city_mpg has implausible max (200) -- implausible as of 2020

,Quantity,Acquisition Value,Total Purchase Cost
count,137680.000000,1.376800e+05,1.376800e+05
mean,15.058687,1.957041e+04,2.724455e+04
std,153.845587,1.592509e+05,3.292570e+05
min,0.000000,0.000000e+00,0.000000e+00
25%,1.000000,6.495000e+01,2.759025e+02
50%,3.000000,3.210000e+02,1.242000e+03
75%,10.000000,2.000000e+03,7.251000e+03
max,40000.000000,2.200000e+07,7.200000e+07


# find duplicates

In [ ]:
duplicateDFRow = concat_mil[concat_mil.duplicated()]
C = concat_mil.copy()
print(C)

# create bool of all duplicate rows = B, so all duplicates = TRUE and will be included in D
# copy = df_copy[['State',	'Station Name (LEA)',	'NSN',	'Item Name', 'UI',	'Acquisition Value',	'DEMIL Code',	'Ship Date']]
B = concat_mil.duplicated(subset=['State',	'Station Name (LEA)',	'NSN',	'Item Name', 'UI',	'Acquisition Value',	'DEMIL Code',	'Ship Date'], keep = False)


# remove duplicates from C using bool, B2
C = C[B == 0]  


# pull out the columns into a new table that are likely duplicates.... # find the duplicates on these columns, = D, D is just duplicates
D = C[B]



# GROUP D into a new table J that has size of all the duplicate columns, qauntity etc
# J2 = D.groupby(['State',	'Station Name (LEA)',	'NSN',	'Item Name',	'UI', 'Acquisition Value',	'DEMIL Code',	'Ship Date'],as_index=False).size().agg({'Quantity': 'first'})
# J = D.loc[D.groupby(['State',	'Station Name (LEA)',	'NSN',	'Item Name',	'UI', 'Acquisition Value',	'DEMIL Code',	'Ship Date'],as_index=False)["State"].idxmin()] # .size().agg({'Quantity': 'first'})

J = D.sort_values(by='State').drop_duplicates(subset=['State',	'Station Name (LEA)',	'NSN',	'Item Name',	'UI', 'Acquisition Value',	'DEMIL Code',	'Ship Date'])
J2 = D.groupby(['State',	'Station Name (LEA)',	'NSN',	'Item Name',	'UI', 'Acquisition Value',	'DEMIL Code',	'Ship Date'],as_index=False).size()

# update J so it represents fully cleaned of duplicates for duplicate rows
J['Quantity'] = J['Quantity'] * J2['size']
J['Total Purchase Cost'] = J['Total Purchase Cost'] * J2['size']


# Merge the new cleaned rows into C, but we also need to remove columns from C that are not needed
df_merge = pd.concat([C, J]).reset_index(drop=True)

# reorganize the rows by state 
# remove all duplicate rows to be replaced in C
# add the size, qauntity of J into C



# rem_dup_pre = pre_ferg_clean.groupby(pre_ferg_clean.columns.tolist(),as_index=False).size()



# rem_dup_pre['Quantity'] = rem_dup_pre['size'] - 1 + rem_dup_pre['Quantity'] 
# rem_dup_pre = rem_dup_pre.drop(['size'], axis=1)
# rem_dup_pre
df_merge

       State  ... Total Purchase Cost
0         AK  ...             3493.00
1         AK  ...            61377.00
2         AK  ...            10000.00
3         AK  ...            10000.00
4         AK  ...            34400.00
...      ...  ...                 ...
137675    WY  ...              234.84
137676    WY  ...              234.84
137677    WY  ...            96466.00
137678    WY  ...           192513.00
137679    WY  ...              966.00

[137680 rows x 10 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,Ship Date,Total Purchase Cost
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",7,Each,499.00,D,2006-05-19,3493.00
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",123,Each,499.00,D,2012-08-30,61377.00
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1385-01-574-4707,"ROBOT,EXPLOSIVE ORDNANCE DISPOSAL",1,Each,10000.00,D,2012-05-24,10000.00
3,AK,ALASKA DEPT OF PUBLIC SAFETY,5820-01-564-4845,"PROCESSOR GROUP,SIGNAL DATA",2,Each,5000.00,D,2012-07-19,10000.00
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,NIGHT VISION GOGGLE,8,Each,4300.00,F,1998-05-19,34400.00
...,...,...,...,...,...,...,...,...,...,...
128041,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,D,2001-11-27 00:00:00.000,234.84
128042,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,D,2002-01-29 00:00:00.000,234.84
128043,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2310-01-111-2274,"TRUCK,AMBULANCE",1,Each,96466.00,C,2019-09-25 00:00:00.000,96466.00
128044,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,"TRUCK,UTILITY",1,Each,192513.00,C,2018-03-12 00:00:00.000,192513.00


# obtain unique types and give each tuple a topic

In [ ]:
 unique_types = df_merge['Item Name'].unique()
 print("size is ", len(unique_types))
 for x in unique_types: 
   print(x)



Streaming output truncated to the last 5000 lines.
TOOL KIT,RIGGING
BIT,SCREWDRIVER
TABLE, HOUSEHOLD
STORAGE CONTAINER,FLEXIBLE
OUTERWEAR, WOMEN'S
RACK,GUN ASSY      
DESC=SCOPE RIFLE  SCOPE LEUPOLD 
DESC=SIGHT RELEX 
TANK-PUMPING UNIT,AIRCRAFT LAVATORY SERVICING,VEHICULAR MOUNTING
FACSIMILE SET
DESC=RADAR GUN 
DESC=NITESSPOT 50 VECTRONIX 
DESC=VIPER SCOPE 
ADAPTER FLIP UP    
DESC=SCOPE NITE VISIONN 
BORESCOPE KIT
EYEWASH,PORTABLE   
CONTAINER,REUSABLE 
CASE,FIELD,OFFICE MACHINE
CHEST,COLLAPSIBLE
POCKET,MAGAZINE    
VIDEO CONFERENCE SYSTEM
DESC=HELMET,RIOT 
DESC=BATON RIOT 
DESC=SHIELD BODY LARGE 
DESC=SHIELD BODY SMALL 
DESC=VEST BODY ARMOR 
BASE VEST,OUTERSHELLL
IMAGER ASSEMBLY,INF
CYLINDER ASSEMBLY,ACTUATING,LINEAR
ROLLER,MOTORIZED
DESC=AIR TRAILER 
ELECTRONIC SHOP,SEMITRAILER MOUNTED
CAMERA SET,TELEVISION
LOUDSPEAKER,CRYSTAL
MOTOR,ALTERNATING CURRENT
CABINET, KITCHEN
FRYER
GRIDDLE
VEGETABLE CUTTING AND SLICING MACHINE
FACESHIELD,RIOT CONTROL
FLATRACK,PALLETIZED LOADING
COMPRESSOR,

In [ ]:
# categorize them
guns = ["GUNS", "RIFLE", "SHOT","CHAIN","SNIPER", "FIREARM", "M249", "RPG7", "TRIGGER", "AUTOMATIC", "22", "ASSAULT", "M59A", "HOLSTER","BARREL" "M4", "M16", "M8",  "PISTOL", "STOCK", "FIRING","REVOLVER", "SCOPE", "LASER", "TARGET", "SILENCE", "CALIBER", ".38", "M24"]
ammo = [ "BULLET","KIT", "9MM", "PROJECTILE", "MAGAZ", "AMMO","ROUND", "ARM", "CARTRIDGE", "MAGAZiNE", "AMMUNITION"]
explosives = ["GRENADE", "MORTAR","EXPLOSIVE", "BOMB", "DYNAMITE", "MISSLE", "MAGAZINE", "40MM"]
armor =  ["SHIELD", "CAMOUFLAGE", "KEVLAR", "ARMOR", "PROTECTIVE", "VEST", "FLAK", "JACKET", "BODY", "HELMET"]
tracking = ["CAM","POSITIONING","RADAR","HEADSETS", "SATELLITE","DETECTOR", "COMMUNICATION","NIGHT", "SURVEILANCE", "INFARED", "TRANS" "PHOTO", "GOGGLE",  "TUNER", "GPS", "RADIO","TARGET", "VISION", "SENSOR", "RECEIVER", "ANTENNA", "SIGNAL", "BINOCULAR", "MINOCULAR"]
riot = ["ASSAULT","INFANTRY", "COMBAT", "BALLISTIC", "CONTROL", "TACTICAL", "STRAP", "TIE", "GAS", "MICS", "STUN", "CLUB", "COMBAT","RECON", "MACE", "MASE", "FLASH","ZIP", "BANG", "HANDCUFFS"]
hand_held_weapons = ["TASER , ""BLADE", "BOXING","KNIFE", "BATON", "WEAPON", "MACHETE"]
food = ["FOOD","BAKING", "PAN", "FRYER", "FAT", "SUGAR", "MICROWAVE", "REFIG"]
medical = ["MED","SLING","TRAUMA", "BLOOD", "PUMP", "SPLINT","STRETCHER", "INTRAVENOUS", "SCISSOR", "DRUG", "AID", "TREATMENT","RAY", "EXAMINAtiON", "DECONTAM","IMAGING", "LIFE", "OXYGEN", "SUPPORT" ,"SURG", "SICK","SLING", "ILL", "BANDAGE", "BREATH","SIRYNGE", "GAUSE", "FIRST", "NEEDLE", "RECOVERY", "MICROSCOPE", "LAB", "INDICATOR", "PATIENT", "VITAL", "DEFIBRILLATOR"]
vehicle = ["VEHICLE", "TRACTOR", "FREIGHT", "MOBILE", "MOTORCYCLE", "CART,", "MINE", "ARTILLERY", "CAR", "JEEP", "HUMVEE", "TANK", "TRUCK", "VEHICULAR", "TURRET"]
aircraft = ["AIRCRAFT","TURBINE", "ROTOR", "WING", "HELI", "PLANE", "DRONE","FLIGHT" ]
advanced_equipment = ["BOT", "REMOTE", "ROBO", "VIRTUAL"]
sea = ["SUBMARINE", "TURBINE", "RUDDER", "SHIP", "NAVY", "KAYAK", "TUG", "HELICOPTER", "ROTARY", "BOAT", "WATER", "SEA", "OCEAN", "SPEED", "SAIL"]

In [ ]:
# add an item_type column
df_merge['Item Type'] = 'misc'
df_merge

,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,Ship Date,Total Purchase Cost,Item Type
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",7,Each,499.00,D,2006-05-19,3493.00,misc
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,"RIFLE,5.56 MILLIMETER",123,Each,499.00,D,2012-08-30,61377.00,misc
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1385-01-574-4707,"ROBOT,EXPLOSIVE ORDNANCE DISPOSAL",1,Each,10000.00,D,2012-05-24,10000.00,misc
3,AK,ALASKA DEPT OF PUBLIC SAFETY,5820-01-564-4845,"PROCESSOR GROUP,SIGNAL DATA",2,Each,5000.00,D,2012-07-19,10000.00,misc
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,NIGHT VISION GOGGLE,8,Each,4300.00,F,1998-05-19,34400.00,misc
...,...,...,...,...,...,...,...,...,...,...,...
128041,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,D,2001-11-27 00:00:00.000,234.84,misc
128042,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,D,2002-01-29 00:00:00.000,234.84,misc
128043,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2310-01-111-2274,"TRUCK,AMBULANCE",1,Each,96466.00,C,2019-09-25 00:00:00.000,96466.00,misc
128044,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,"TRUCK,UTILITY",1,Each,192513.00,C,2018-03-12 00:00:00.000,192513.00,misc


In [ ]:
# what values contain 'guns'
# bool_guns = concat_mil['Item Name'].str.contains('GUNS')



for x in tracking:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "tracking"


for x in riot:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "riot"

for x in hand_held_weapons:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "short_distance_weapons"

for x in food:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "food"

for x in medical:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "medical"


for x in vehicle:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "vehicle"


for x in aircraft:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "aircraft"


for x in advanced_equipment:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "advanced_equipment"

for x in sea:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "sea"

for x in ammo:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "ammo"

for x in explosives:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "explosives"

for x in guns:
  bool = df_merge['Item Name'].str.contains(x)
  df_merge['Item Type'][bool] = "gun"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A 


LIST OF ITEMS CATEGORIES:
* tracking
* riot
* short_distance_weapons
* food
* medical
* vehicle
* aircraft
* advanced_equipment
* sea
* ammo
* explosives
* gun


DEMIL_codes
-- also provide a way to obtain military grade vs. civilian
CIVILIAN: A, B, Q
MILITARY / VIOLENT = G, P, F, D, C, E


# add in a year column so we can do joins more effectively

In [ ]:
df_merge['year'] = df_merge['Ship Date'].str.slice(0,4)

# Reorder columns in dataframe

In [ ]:
# reorder columns
df = df_merge.reindex(columns = ['State', 'Station Name (LEA)', 'NSN', 'year', 'Ship Date', 'Item Type','Item Name','Quantity', 'UI', 'Acquisition Value','Total Purchase Cost','DEMIL Code'])

In [ ]:
df

,State,Station Name (LEA),NSN,year,Ship Date,Item Type,Item Name,Quantity,UI,Acquisition Value,Total Purchase Cost,DEMIL Code
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,2006,2006-05-19,gun,"RIFLE,5.56 MILLIMETER",7,Each,499.00,3493.00,D
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,2012,2012-08-30,gun,"RIFLE,5.56 MILLIMETER",123,Each,499.00,61377.00,D
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1385-01-574-4707,2012,2012-05-24,explosives,"ROBOT,EXPLOSIVE ORDNANCE DISPOSAL",1,Each,10000.00,10000.00,D
3,AK,ALASKA DEPT OF PUBLIC SAFETY,5820-01-564-4845,2012,2012-07-19,tracking,"PROCESSOR GROUP,SIGNAL DATA",2,Each,5000.00,10000.00,D
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,1998,1998-05-19,tracking,NIGHT VISION GOGGLE,8,Each,4300.00,34400.00,F
...,...,...,...,...,...,...,...,...,...,...,...,...
128041,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,2001,2001-11-27 00:00:00.000,gun,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,234.84,D
128042,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,2002,2002-01-29 00:00:00.000,gun,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,234.84,D
128043,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2310-01-111-2274,2019,2019-09-25 00:00:00.000,vehicle,"TRUCK,AMBULANCE",1,Each,96466.00,96466.00,C
128044,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,2018,2018-03-12 00:00:00.000,vehicle,"TRUCK,UTILITY",1,Each,192513.00,192513.00,C


# rename columns

In [ ]:
df = df.rename(columns = {'State':"state", 'Station Name (LEA)':"station_name", "year":"acq_year", 'Ship Date':'ship_date', 'Item Type':'item_type','Item Name':'item_name','Quantity':'quantity', 'Acquisition Value':'cost_per_item','Total Purchase Cost':'total_purchase_cost','DEMIL Code': 'DEMIL_code'} )

In [ ]:
df

,state,station_name,NSN,acq_year,ship_date,item_type,item_name,quantity,UI,cost_per_item,total_purchase_cost,DEMIL_code
0,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,2006,2006-05-19,gun,"RIFLE,5.56 MILLIMETER",7,Each,499.00,3493.00,D
1,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,2012,2012-08-30,gun,"RIFLE,5.56 MILLIMETER",123,Each,499.00,61377.00,D
2,AK,ALASKA DEPT OF PUBLIC SAFETY,1385-01-574-4707,2012,2012-05-24,explosives,"ROBOT,EXPLOSIVE ORDNANCE DISPOSAL",1,Each,10000.00,10000.00,D
3,AK,ALASKA DEPT OF PUBLIC SAFETY,5820-01-564-4845,2012,2012-07-19,tracking,"PROCESSOR GROUP,SIGNAL DATA",2,Each,5000.00,10000.00,D
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,1998,1998-05-19,tracking,NIGHT VISION GOGGLE,8,Each,4300.00,34400.00,F
...,...,...,...,...,...,...,...,...,...,...,...,...
128041,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,2001,2001-11-27 00:00:00.000,gun,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,234.84,D
128042,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,1005-00-726-5655,2002,2002-01-29 00:00:00.000,gun,"PISTOL,CALIBER .45,AUTOMATIC",4,Each,58.71,234.84,D
128043,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2310-01-111-2274,2019,2019-09-25 00:00:00.000,vehicle,"TRUCK,AMBULANCE",1,Each,96466.00,96466.00,C
128044,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,2018,2018-03-12 00:00:00.000,vehicle,"TRUCK,UTILITY",1,Each,192513.00,192513.00,C


# sort by state and ship date

In [ ]:
df = df.sort_values(by=['state', 'ship_date'])
df

,state,station_name,NSN,acq_year,ship_date,item_type,item_name,quantity,UI,cost_per_item,total_purchase_cost,DEMIL_code
77467,AK,HAINES BOROUGH POLICE DEPT,1005-00-589-1271,1994,1994-07-19 00:00:00.000,gun,"RIFLE,7.62 MILLIMETER",5,Each,138.00,690.00,D
77469,AK,JUNEAU POLICE DEPARTMENT,1005-00-589-1271,1994,1994-08-08 00:00:00.000,gun,"RIFLE,7.62 MILLIMETER",4,Each,138.00,552.00,D
77468,AK,JUNEAU POLICE DEPARTMENT,1005-00-073-9421,1998,1998-01-21 00:00:00.000,gun,"RIFLE,5.56 MILLIMETER",10,Each,499.00,4990.00,D
77404,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,1998,1998-01-22 00:00:00.000,gun,"RIFLE,5.56 MILLIMETER",56,Each,499.00,27944.00,D
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,1998,1998-05-19,tracking,NIGHT VISION GOGGLE,8,Each,4300.00,34400.00,F
...,...,...,...,...,...,...,...,...,...,...,...,...
77375,WY,US DHS ICE CHEYENNE,8465-01-393-6515,2015,2015-02-05,misc,"MAT,SLEEPING,SELF-INFLATING",2,Each,47.09,94.18,A
77380,WY,US DHS ICE CHEYENNE,8465-01-524-5285,2015,2015-02-05,misc,RUCKSACK LARGE FIELD PACK,3,Each,83.36,250.08,A
77381,WY,US DHS ICE CHEYENNE,8970-00-149-1094,2015,2015-03-03,misc,"MEAL,INDIVIDUAL",5,Box,100.28,100.28,A
128044,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,2018,2018-03-12 00:00:00.000,vehicle,"TRUCK,UTILITY",1,Each,192513.00,192513.00,C


In [ ]:
df.isnull().any()

state                  False
station_name           False
NSN                    False
acq_year               False
ship_date              False
item_type              False
item_name              False
quantity               False
UI                     False
cost_per_item          False
total_purchase_cost    False
DEMIL_code             False
dtype: bool

# drop uncessary columns

In [ ]:
df2 = df.drop('ship_date', axis=1)
# df2 = df.drop('UI', axis=1)
# df = df.drop('DEMIL_code', axis=1)
# df2 = df.drop('NSN', axis=1)
df2

,state,station_name,NSN,acq_year,item_type,item_name,quantity,UI,cost_per_item,total_purchase_cost,DEMIL_code
77467,AK,HAINES BOROUGH POLICE DEPT,1005-00-589-1271,1994,gun,"RIFLE,7.62 MILLIMETER",5,Each,138.00,690.00,D
77469,AK,JUNEAU POLICE DEPARTMENT,1005-00-589-1271,1994,gun,"RIFLE,7.62 MILLIMETER",4,Each,138.00,552.00,D
77468,AK,JUNEAU POLICE DEPARTMENT,1005-00-073-9421,1998,gun,"RIFLE,5.56 MILLIMETER",10,Each,499.00,4990.00,D
77404,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,1998,gun,"RIFLE,5.56 MILLIMETER",56,Each,499.00,27944.00,D
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,1998,tracking,NIGHT VISION GOGGLE,8,Each,4300.00,34400.00,F
...,...,...,...,...,...,...,...,...,...,...,...
77375,WY,US DHS ICE CHEYENNE,8465-01-393-6515,2015,misc,"MAT,SLEEPING,SELF-INFLATING",2,Each,47.09,94.18,A
77380,WY,US DHS ICE CHEYENNE,8465-01-524-5285,2015,misc,RUCKSACK LARGE FIELD PACK,3,Each,83.36,250.08,A
77381,WY,US DHS ICE CHEYENNE,8970-00-149-1094,2015,misc,"MEAL,INDIVIDUAL",5,Box,100.28,100.28,A
128044,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,2018,vehicle,"TRUCK,UTILITY",1,Each,192513.00,192513.00,C


# remove all commas inside of columns

In [ ]:
# df2['item_name'] = df2.item_name.str.strip(',')

df2['item_name'] = df2.item_name.replace(',',' ', regex=True)
df2

,state,station_name,NSN,acq_year,item_type,item_name,quantity,UI,cost_per_item,total_purchase_cost,DEMIL_code
77467,AK,HAINES BOROUGH POLICE DEPT,1005-00-589-1271,1994,gun,RIFLE 7.62 MILLIMETER,5,Each,138.00,690.00,D
77469,AK,JUNEAU POLICE DEPARTMENT,1005-00-589-1271,1994,gun,RIFLE 7.62 MILLIMETER,4,Each,138.00,552.00,D
77468,AK,JUNEAU POLICE DEPARTMENT,1005-00-073-9421,1998,gun,RIFLE 5.56 MILLIMETER,10,Each,499.00,4990.00,D
77404,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,1998,gun,RIFLE 5.56 MILLIMETER,56,Each,499.00,27944.00,D
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,1998,tracking,NIGHT VISION GOGGLE,8,Each,4300.00,34400.00,F
...,...,...,...,...,...,...,...,...,...,...,...
77375,WY,US DHS ICE CHEYENNE,8465-01-393-6515,2015,misc,MAT SLEEPING SELF-INFLATING,2,Each,47.09,94.18,A
77380,WY,US DHS ICE CHEYENNE,8465-01-524-5285,2015,misc,RUCKSACK LARGE FIELD PACK,3,Each,83.36,250.08,A
77381,WY,US DHS ICE CHEYENNE,8970-00-149-1094,2015,misc,MEAL INDIVIDUAL,5,Box,100.28,100.28,A
128044,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,2018,vehicle,TRUCK UTILITY,1,Each,192513.00,192513.00,C


# recombine similar rows
since we remove ship date we will now recombine duplicates

In [ ]:
# duplicateDFRow = concat_mil[concat_mil.duplicated()]
C = df2.copy()

# create bool of all duplicate rows = B, so all duplicates = TRUE and will be included in D
# copy = df_copy[['State',	'Station Name (LEA)',	'NSN',	'Item Name', 'UI',	'Acquisition Value',	'DEMIL Code',	'Ship Date']]
B = df2.duplicated(subset=['state',	'station_name',	'NSN',	'acq_year',	'item_type',	'item_name',	'UI',	'cost_per_item',	'DEMIL_code'], keep = False)


# remove duplicates from C using bool, B2
C = C[B == 0]  


# pull out the columns into a new table that are likely duplicates.... # find the duplicates on these columns, = D, D is just duplicates
D = C[B]



# GROUP D into a new table J that has size of all the duplicate columns, qauntity etc
# J2 = D.groupby(['State',	'Station Name (LEA)',	'NSN',	'Item Name',	'UI', 'Acquisition Value',	'DEMIL Code',	'Ship Date'],as_index=False).size().agg({'Quantity': 'first'})
# J = D.loc[D.groupby(['State',	'Station Name (LEA)',	'NSN',	'Item Name',	'UI', 'Acquisition Value',	'DEMIL Code',	'Ship Date'],as_index=False)["State"].idxmin()] # .size().agg({'Quantity': 'first'})

J = D.sort_values(by='state').drop_duplicates(subset=['state',	'station_name',	'NSN',	'acq_year',	'item_type',	'item_name',	'UI',	'cost_per_item',	'DEMIL_code'])
J2 = D.groupby(['state',	'station_name',	'NSN',	'acq_year',	'item_type',	'item_name',	'UI',	'cost_per_item',	'DEMIL_code'],as_index=False).size()

# update J so it represents fully cleaned of duplicates for duplicate rows
J['quantity'] = J['quantity'] * J2['size']
J['total_purchase_cost'] = J['total_purchase_cost'] * J2['size']


# Merge the new cleaned rows into C, but we also need to remove columns from C that are not needed
df_merge2 = pd.concat([C, J]).reset_index(drop=True)

# reorganize the rows by state 
# remove all duplicate rows to be replaced in C
# add the size, qauntity of J into C



# rem_dup_pre = pre_ferg_clean.groupby(pre_ferg_clean.columns.tolist(),as_index=False).size()



# rem_dup_pre['Quantity'] = rem_dup_pre['size'] - 1 + rem_dup_pre['Quantity'] 
# rem_dup_pre = rem_dup_pre.drop(['size'], axis=1)
# rem_dup_pre
df_merge2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,state,station_name,NSN,acq_year,item_type,item_name,quantity,UI,cost_per_item,total_purchase_cost,DEMIL_code
0,AK,HAINES BOROUGH POLICE DEPT,1005-00-589-1271,1994,gun,RIFLE 7.62 MILLIMETER,5,Each,138.00,690.00,D
1,AK,JUNEAU POLICE DEPARTMENT,1005-00-589-1271,1994,gun,RIFLE 7.62 MILLIMETER,4,Each,138.00,552.00,D
2,AK,JUNEAU POLICE DEPARTMENT,1005-00-073-9421,1998,gun,RIFLE 5.56 MILLIMETER,10,Each,499.00,4990.00,D
3,AK,ALASKA DEPT OF PUBLIC SAFETY,1005-00-073-9421,1998,gun,RIFLE 5.56 MILLIMETER,56,Each,499.00,27944.00,D
4,AK,ALASKA DEPT OF PUBLIC SAFETY,5855-01-228-0936,1998,tracking,NIGHT VISION GOGGLE,8,Each,4300.00,34400.00,F
...,...,...,...,...,...,...,...,...,...,...,...
105499,WY,US DHS ICE CHEYENNE,8465-01-393-6515,2015,misc,MAT SLEEPING SELF-INFLATING,2,Each,47.09,94.18,A
105500,WY,US DHS ICE CHEYENNE,8465-01-524-5285,2015,misc,RUCKSACK LARGE FIELD PACK,3,Each,83.36,250.08,A
105501,WY,US DHS ICE CHEYENNE,8970-00-149-1094,2015,misc,MEAL INDIVIDUAL,5,Box,100.28,100.28,A
105502,WY,WASHAKIE COUNTY SHERIFF'S OFFICE,2320-01-413-3739,2018,vehicle,TRUCK UTILITY,1,Each,192513.00,192513.00,C


In [ ]:
# check unique types
df_merge2.UI.unique()

array(['Each', 'Pair', 'Kit', 'Set', 'Box', 'Assembly', 'Plate', 'Sheet',
       'Foot', 'Roll', 'Package', 'Coil', 'Unknown', 'Dozen', 'Reel',
       'Spool', 'Thousand', 'Bag', 'Hundred', 'Pound', 'Board Foot',
       'Yard', 'Tube', 'Lot', 'Packet', 'Can', 'Drum', 'Length', 'Outfit',
       'Bundle', 'Bale', 'Case', 'Assortment', 'Square Foot', 'Gross',
       'Pad', 'Cone', 'Bottle', 'Strip', 'Cartridge', 'Quart', 'Ounce',
       'Book', 'Inch', 'Meter', 'Gallon', 'Pint', 'Container'],
      dtype=object)

In [ ]:
df_merge2.update('"' + df_merge2.astype(str) + '"')
df_merge2

,state,station_name,NSN,acq_year,item_type,item_name,quantity,UI,cost_per_item,total_purchase_cost,DEMIL_code
0,"""AK""","""HAINES BOROUGH POLICE DEPT""","""1005-00-589-1271""","""1994""","""gun""","""RIFLE 7.62 MILLIMETER""","""5""","""Each""","""138.0""","""690.0""","""D"""
1,"""AK""","""JUNEAU POLICE DEPARTMENT""","""1005-00-589-1271""","""1994""","""gun""","""RIFLE 7.62 MILLIMETER""","""4""","""Each""","""138.0""","""552.0""","""D"""
2,"""AK""","""JUNEAU POLICE DEPARTMENT""","""1005-00-073-9421""","""1998""","""gun""","""RIFLE 5.56 MILLIMETER""","""10""","""Each""","""499.0""","""4990.0""","""D"""
3,"""AK""","""ALASKA DEPT OF PUBLIC SAFETY""","""1005-00-073-9421""","""1998""","""gun""","""RIFLE 5.56 MILLIMETER""","""56""","""Each""","""499.0""","""27944.0""","""D"""
4,"""AK""","""ALASKA DEPT OF PUBLIC SAFETY""","""5855-01-228-0936""","""1998""","""tracking""","""NIGHT VISION GOGGLE""","""8""","""Each""","""4300.0""","""34400.0""","""F"""
...,...,...,...,...,...,...,...,...,...,...,...
105499,"""WY""","""US DHS ICE CHEYENNE""","""8465-01-393-6515""","""2015""","""misc""","""MAT SLEEPING SELF-INFLATING""","""2""","""Each""","""47.09""","""94.18""","""A"""
105500,"""WY""","""US DHS ICE CHEYENNE""","""8465-01-524-5285""","""2015""","""misc""","""RUCKSACK LARGE FIELD PACK""","""3""","""Each""","""83.36""","""250.07999999999998""","""A"""
105501,"""WY""","""US DHS ICE CHEYENNE""","""8970-00-149-1094""","""2015""","""misc""","""MEAL INDIVIDUAL""","""5""","""Box""","""100.28""","""100.28""","""A"""
105502,"""WY""","""WASHAKIE COUNTY SHERIFF'S OFFICE""","""2320-01-413-3739""","""2018""","""vehicle""","""TRUCK UTILITY""","""1""","""Each""","""192513.0""","""192513.0""","""C"""


# download CSV files

In [ ]:

df_merge2.to_csv("military_transfers.csv", index=True)
# rem_dup_po.to_csv("postFerg.csv", index=False)
df_merge2.index.name = "id"
# census.to_csv("census.csv", index=True)

from google.colab import files
files.download('military_transfers.csv') 


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>